In [ ]:
!pip install wget
!pip install numpy
!pip install Cython

In [ ]:
import os

!mkdir {os.path.join("dataset", "train")}
!mkdir {os.path.join("dataset", "test")}
!mkdir {os.path.join("dependencies", "preTrainedModels")}
!mkdir {os.path.join("dependencies", "object_detection")}
!mkdir {os.path.join("dependencies", "protoc")}
!mkdir {os.path.join("dependencies", "tensorflowRecords")}

In [ ]:
!git clone https://github.com/tensorflow/models dependencies/object_detection

In [ ]:
!pip install tensorflow
!pip install scipy
!pip install tensorflow_io
!pip install matplotlib
!pip install tf-models-official
!pip install pytz
!pip install gin-config==0.1.1

In [ ]:
import wget
import os

url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)
!move protoc-3.15.6-win64.zip dependencies/protoc
!cd dependencies/protoc && tar -xf protoc-3.15.6-win64.zip
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join("dependencies", "protoc", 'bin'))
!cd dependencies/object_detection/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd dependencies/object_detection/research/slim && pip install -e .

In [ ]:
# Deve aparecer "OK (skipped=1)" no final
!python dependencies/object_detection/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
import object_detection

In [ ]:
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
preTrainedModelDownloadLink = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
preTrainedModelName = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
folderName = 'dependencies/preTrainedModels/' + preTrainedModelName

In [ ]:
import wget

wget.download(preTrainedModelDownloadLink)

!cd dependencies/preTrainedModels && mkdir {preTrainedModelName}
!move {preTrainedModelName + '.tar.gz'} {'dependencies/preTrainedModels/' + preTrainedModelName + '/' + preTrainedModelName + '.tar.gz'}
!cd {'dependencies/preTrainedModels/' + preTrainedModelName} && tar -zxvf {preTrainedModelName + '.tar.gz'}

In [ ]:
labels = [{'name':'soy', 'id':1}, {'name':'pod', 'id':2}]

with open('label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
!git clone https://github.com/nicknochnack/GenerateTFRecord {"dependencies/tensorflowRecords"}

In [ ]:
!python {"dependencies/tensorflowRecords/generate_tfrecord.py"} -x {"dataset/train"} -l {"label_map.pbtxt"} -o {"dataset/train.record"}
!python {"dependencies/tensorflowRecords/generate_tfrecord.py"} -x {"dataset/test"} -l {"label_map.pbtxt"} -o {"dataset/test.record"}

In [ ]:
!copy {os.path.join("dependencies", "preTrainedModels", preTrainedModelName, preTrainedModelName, "pipeline.config")} {os.path.join("models", preTrainedModelName)}

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
configurationFile = os.path.join("models", preTrainedModelName, "pipeline.config")
labelMap = 'label_map.pbtxt'
preTrainedModelPath = os.path.join('dependencies', 'preTrainedModels', preTrainedModelName, preTrainedModelName)
labels = [{'name':'soy', 'id':1}, {'name':'pod', 'id':2}]

config = config_util.get_configs_from_pipeline_file(configurationFile)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(configurationFile, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)
    
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 2
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(preTrainedModelPath, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= labelMap
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join("dataset", 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = labelMap
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join("dataset", 'test.record')]

config_text = text_format.MessageToString(pipeline_config)

with tf.io.gfile.GFile(configurationFile, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [ ]:
trainingScriptPath = os.path.join("dependencies", "object_detection", 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
# Copie o comando imprimido e abra um CMD no diretório notebooks, ative a venv e execute o comando
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=500".format(trainingScriptPath, os.path.join("models", preTrainedModelName), configurationFile)

print(command)

In [ ]:
# Copie o comando imprimido e abra um CMD no diretório notebooks, ative a venv e execute o comando

command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(trainingScriptPath, os.path.join("models", preTrainedModelName), configurationFile, os.path.join("models", preTrainedModelName))

print(command)